In [4]:
from transformers import DebertaV2Tokenizer, AutoTokenizer, DebertaV2ForMaskedLM
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset, load_from_disk
from tqdm.auto import tqdm

c:\Users\Matheus\Documents\Git\Longformer-PT\venv_lg_former\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
dataset = load_from_disk("c4ai-wik")

In [24]:
tokenizer = AutoTokenizer.from_pretrained("deberta_pt_tokenizer")

In [7]:
model = DebertaV2ForMaskedLM.from_pretrained("microsoft/deberta-v3-base", vocab_size=tokenizer.vocab_size, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForMaskedLM: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

### Splitting data by max_lenght

In [10]:
MAX_POS_EMBD = model.config.max_position_embeddings
WINDOW = 200

In [72]:
from functools import partial

def _tokenizer(sample, tokenizer, window, max_lenght):
    print(len(sample['text']))

    tkn_text = tokenizer(str(sample['text']), add_special_tokens=False)

    dev_train_dataset = []
    for i in range(0, len(tkn_text['input_ids']), window):
        dev_train_dataset.append(tokenizer.decode(tkn_text['input_ids'][i:max_lenght+i]))
    return {k: v for k, v in enumerate(dev_train_dataset)}

partial_tokenizer = partial(_tokenizer, tokenizer=tokenizer, window=WINDOW, max_lenght=MAX_POS_EMBD-2)
# partial_tokenizer = lambda i: _tokenizer(i, tokenizer=tokenizer, window=WINDOW, max_lenght=MAX_POS_EMBD-2)

In [73]:
tokenizer.decode

<bound method PreTrainedTokenizerBase.decode of DebertaTokenizerFast(name_or_path='deberta_pt_tokenizer', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)>

In [74]:
partial_tokenizer({'text': 'meu nome eh matheus'})

19


{0: 'meu nome eh matheus'}

In [75]:
type(dataset)

datasets.arrow_dataset.Dataset

In [76]:
dataset = dataset.map(partial_tokenizer, batched=True)#, num_proc=4)

Map:   0%|          | 0/960139 [00:00<?, ? examples/s]

1000


TypeError: Provided `function` which is applied to all elements of table returns a `dict` of types [<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, 

In [ ]:
# data_collator = DataCollatorWithPadding(
#     tokenizer=tok, padding="max_length", max_length=MAX_LENGTH
# )

In [ ]:
# train_loader = DataLoader(DatasetKGC(train), batch_size=BATCH_SIZE, shuffle=False)